In [1]:
%matplotlib inline
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("titanic").getOrCreate()

In [2]:
train = spark.read.csv('./train.csv', header="true", inferSchema="true")
test = spark.read.csv('./test.csv', header="true", inferSchema="true")

train.printSchema()
test.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [3]:
train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [4]:
labelCol = 'Survived'
train.groupby(labelCol).count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [5]:
train.crosstab(labelCol, 'Sex').show()

+------------+------+----+
|Survived_Sex|female|male|
+------------+------+----+
|           1|   233| 109|
|           0|    81| 468|
+------------+------+----+



In [6]:
train.crosstab(labelCol, 'Pclass').show()

+---------------+---+---+---+
|Survived_Pclass|  1|  2|  3|
+---------------+---+---+---+
|              1|136| 87|119|
|              0| 80| 97|372|
+---------------+---+---+---+



In [7]:
from pyspark.sql.functions import col

# Find columns with missing values
def findNullColumns(df):
    nullCols = []
    numRows = df.count()
    for k in df.columns:
        notNullCount = df.filter(col(k).isNotNull()).count()
        if df.filter(col(k).isNotNull()).count() != numRows:
            nullCols.append((k, notNullCount/numRows))
    return nullCols

findNullColumns(train)

[('Age', 0.8013468013468014),
 ('Cabin', 0.22895622895622897),
 ('Embarked', 0.9977553310886644)]

In [8]:
# Find percentage of data missing values

train.filter(train.Age.isNull()).count() / train.count()


0.19865319865319866

In [9]:
train = train.filter(train.Age.isNotNull())
train.count()

714

In [10]:
train.filter(train.Embarked.isNull()).count() / train.count()
train = train.filter(train.Embarked.isNotNull())
train.count()

712

In [11]:
train.filter(train.Cabin.isNull()).count() / train.count()
train = train.drop('Cabin')
train.count()

712

In [12]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [13]:
train = train.drop('Cabin')
train

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Embarked: string]

In [14]:
from pyspark.ml.feature import StringIndexer

embarkedIndexer = StringIndexer(inputCol='Embarked', outputCol='indexedEmbarked')
sexFeatureIndexer = StringIndexer(inputCol='Sex', outputCol='indexedSex')

In [15]:
from pyspark.ml.feature import Bucketizer

ageSplits = [0, 16, 32, 48, 64, 200]
ageBucketizer = Bucketizer(splits=ageSplits, inputCol='Age', outputCol='bucketedAge')
fareSplits = [-float('inf'), 7.91, 14.454, 31, float('inf')]
fareBucketizer = Bucketizer(splits=fareSplits, inputCol='Fare', outputCol='bucketedFare')

# train_cleaned = ageBucketizer.transform(train).drop('Age')
# train_cleaned = fareBucketizer.transform(train_cleaned).drop('Fare')
# train_cleaned.printSchema()

In [16]:
from pyspark.ml.feature import OneHotEncoderEstimator, VectorIndexer

oneHotEncoderEstimator = OneHotEncoderEstimator(inputCols=['indexedSex', 'indexedEmbarked', 'bucketedFare', 'bucketedAge'], 
                                                outputCols=['oneHotSex', 'oneHotEmbarked','oneHotFare', 'oneHotAge'])
# oneHotEncoderEstimator = oneHotEncoderEstimator.fit(train_cleaned)                                            
# train_cleaned = oneHotEncoderEstimator.transform(train_cleaned)


In [17]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch', 'bucketedAge', 'bucketedFare', 'indexedSex'], outputCol='features')
# train_cleaned = assembler.transform(train_cleaned)
# cols = ['Pclass', 'SibSp', 'Parch', 'bucketedAge', 'bucketedFare', 'indexedSex']
# train_cleaned = train_cleaned.drop(*cols)
# train_cleaned.show()

In [18]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol=labelCol, featuresCol='features')

In [19]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[embarkedIndexer, sexFeatureIndexer, ageBucketizer, 
                            fareBucketizer, oneHotEncoderEstimator, assembler, rf])

grid = ParamGridBuilder().addGrid(rf.numTrees, [15, 20, 25, 30]).addGrid(rf.maxDepth, [3, 5]).build()

cv = CrossValidator(estimator=pipeline, 
                    estimatorParamMaps=grid, 
                    evaluator=BinaryClassificationEvaluator(labelCol='Survived'), 
                    numFolds=10)
model = cv.fit(train)
train = model.transform(train)

In [20]:
evaluator = model.getEvaluator()
evaluator.evaluate(train)

0.8996495020964361

In [21]:
test = model.transform(test)
test.show()

Py4JJavaError: An error occurred while calling o51372.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2373.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2373.0 (TID 2865, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<Pclass_double_VectorAssembler_4d05a703fc0d3ad73a9a:double,SibSp_double_VectorAssembler_4d05a703fc0d3ad73a9a:double,Parch_double_VectorAssembler_4d05a703fc0d3ad73a9a:double,bucketedAge:double,bucketedFare:double,indexedSex:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(Spark